In [1]:
import pandas as pd
import math
from ast import literal_eval

In [2]:

orig = pd.read_csv("../raw/usa_with_details.csv")
orig_n = orig.shape[0]
# todo : scams => company name , //orgs , ?// wfh single Node , //states , ? country names mentioned.

In [3]:
cnt = []
for i in range(orig_n):
    if not isinstance(orig["Description"][i],str):
        orig["Description"][i] = ""
        cnt.append(i)
cnt

/home/sakshamm/environments/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[5222, 16686]

In [4]:
spec_chars = ["&","*","[","]","^","_","{","|","}",";","\n","\r","rsquo","rdquo","ldquo",
              r"<.*?>", #html
              r'^[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+$1', # email
              r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", #url
              r"^(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]))\1|(?:(?:29|30)(\/|-|\.)(?:0?[13-9]|1[0-2])\2))(?:(?:1[6-9]|[2-9]\d)?\d{2})$|^(?:29(\/|-|\.)0?2\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|^(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9])|(?:1[0-2]))\4(?:(?:1[6-9]|[2-9]\d)?\d{2})$",#date
            ]
for char in spec_chars:
    orig["Description"] = orig["Description"].str.replace(char, ' ')

In [5]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


In [6]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/sakshamm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sakshamm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [9]:
lda_prepared_tokens=[]
original = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    original.append(line)
    tokens = prepare_text_for_lda(line)
    lda_prepared_tokens.append(tokens)

In [15]:
from datetime import datetime 
def isBetween (date,start,end) :
    start = datetime.strptime(start, "%d-%m-%Y")
    end = datetime.strptime(end, "%d-%m-%Y")
    return start <= date <= end

In [215]:
text_data = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"01-08-2018","31-12-2018"):
        tokens = lda_prepared_tokens[fr]
        text_data.append(tokens)
len(text_data)

2061

In [216]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [217]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"email" + 0.030*"contact" + 0.024*"company" + 0.022*"receive"')
(1, '0.022*"quality" + 0.021*"shipment" + 0.020*"phone" + 0.020*"email"')
(2, '0.031*"ask" + 0.026*"office" + 0.025*"would" + 0.019*"people"')
(3, '0.018*"good" + 0.014*"would" + 0.013*"month" + 0.013*"training"')
(4, '0.027*"company" + 0.019*"email" + 0.017*"offer" + 0.017*"position"')
(5, '0.027*"state" + 0.025*"website" + 0.021*"want" + 0.018*"number"')
(6, '0.018*"check" + 0.017*"would" + 0.017*"email" + 0.016*"interview"')
(7, '0.047*"cards" + 0.035*"check" + 0.033*"assignment" + 0.025*"would"')
(8, '0.064*"check" + 0.032*"receive" + 0.031*"email" + 0.026*"deposit"')
(9, '0.064*"online" + 0.039*"amazon" + 0.033*"resume" + 0.032*"hire"')


In [218]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [196]:
text_data = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"01-01-2015","31-12-2015"):
        tokens = lda_prepared_tokens[fr]
        text_data.append(tokens)
len(text_data)

796

In [197]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [198]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"company" + 0.011*"training" + 0.011*"benard" + 0.009*"office"')
(1, '0.013*"company" + 0.013*"would" + 0.012*"business" + 0.011*"email"')
(2, '0.043*"check" + 0.039*"money" + 0.019*"receive" + 0.015*"shopper"')
(3, '0.017*"email" + 0.015*"assignment" + 0.015*"receive" + 0.012*"check"')
(4, '0.020*"company" + 0.015*"brian" + 0.015*"working" + 0.013*"software"')
(5, '0.038*"check" + 0.027*"email" + 0.021*"company" + 0.020*"would"')
(6, '0.013*"email" + 0.012*"contact" + 0.009*"company" + 0.008*"check"')
(7, '0.029*"package" + 0.013*"company" + 0.012*"email" + 0.012*"receive"')
(8, '0.026*"company" + 0.018*"receive" + 0.013*"payment" + 0.012*"email"')
(9, '0.031*"company" + 0.015*"employee" + 0.012*"would" + 0.012*"call"')


In [199]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
text_data = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"01-01-2016","31-12-2016"):
        tokens = lda_prepared_tokens[fr]
        text_data.append(tokens)
len(text_data)

In [188]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [189]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.018*"employee" + 0.013*"money" + 0.012*"email" + 0.011*"nicole"')
(1, '0.024*"company" + 0.021*"interview" + 0.015*"email" + 0.014*"check"')
(2, '0.023*"email" + 0.012*"account" + 0.011*"address" + 0.010*"company"')
(3, '0.066*"check" + 0.031*"money" + 0.024*"receive" + 0.020*"email"')
(4, '0.025*"hellip" + 0.014*"check" + 0.013*"company" + 0.012*"2/2/16"')
(5, '0.011*"email" + 0.011*"position" + 0.009*"receive" + 0.008*"provide"')
(6, '0.020*"package" + 0.018*"company" + 0.012*"receive" + 0.012*"working"')
(7, '0.024*"would" + 0.024*"email" + 0.019*"company" + 0.018*"receive"')
(8, '0.013*"7/26/2016" + 0.010*"offer" + 0.009*"vince" + 0.008*"description"')
(9, '0.012*"number" + 0.011*"receive" + 0.009*"contact" + 0.009*"would"')


In [190]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [249]:
text_data = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"24-04-2020","01-10-2020"):
        tokens = lda_prepared_tokens[fr]
        text_data.append(tokens)
len(text_data)

1538

In [251]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [257]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=10)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.025*"company" + 0.025*"email" + 0.018*"interview" + 0.015*"would"')
(1, '0.018*"company" + 0.012*"email" + 0.011*"position" + 0.010*"training"')
(2, '0.024*"check" + 0.019*"email" + 0.017*"receive" + 0.015*"company"')
(3, '0.035*"check" + 0.017*"would" + 0.015*"email" + 0.012*"deposit"')
(4, '0.035*"package" + 0.020*"receive" + 0.015*"company" + 0.012*"email"')


In [267]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [13]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file
from contextualized_topic_models.datasets.dataset import CTMDataset


qt = TopicModelDataPreparation("distiluse-base-multilingual-cased")



In [17]:
unCovid = []
preCovidTokens = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"24-04-2020","01-10-2020"):
        un = original[fr]
        unCovid.append(un)
        preCovidTokens.append(" ".join(lda_prepared_tokens[fr]))
print(len(unCovid))
print(len(preCovidTokens))
text_for_contextual = unCovid
text_for_bow = preCovidTokens

training_dataset = qt.create_training_set(text_for_contextual, text_for_bow)

ctm = ZeroShotTM(input_size=len(qt.vocab), bert_input_size=512, n_components=5)

ctm.fit(training_dataset) # run the model

ctm.get_topics()

1538
1538


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [153800/153800]	Train Loss: 555.8497379536127	Time: 0:00:00.403511: : 100it [00:41,  2.41it/s]


defaultdict(list,
            {0: ['standby',
              'gxc1820td',
              'multifunction',
              '2pm',
              'isolate',
              'duplex',
              'crosscut',
              'mc2535adwe',
              'denise',
              'disappointment'],
             1: ['disappear',
              'standby',
              'authorization',
              'disappointment',
              '2pm',
              'denise',
              'isolate',
              'gxc1820td',
              'multifunction',
              'daichi'],
             2: ['ask',
              'call',
              'website',
              'found',
              'people',
              'everything',
              'recruiter',
              'suppose',
              'number',
              'person'],
             3: ['company',
              'receive',
              'would',
              'position',
              'question',
              'address',
              'interview',
              'tr

In [18]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file
from contextualized_topic_models.datasets.dataset import CTMDataset

qt = TopicModelDataPreparation("bert-base-nli-mean-tokens")

In [25]:
unCovid = []
preCovidTokens = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    date = datetime.strptime(orig["c"][fr],"%b %d, %Y")
    if  isBetween(date,"01-08-2018","31-12-2018"):
        un = original[fr]
        unCovid.append(un)
        preCovidTokens.append(" ".join(lda_prepared_tokens[fr]))
print(len(unCovid))
print(len(preCovidTokens))
list_of_unpreprocessed_documents = unCovid
list_of_preprocessed_documents = preCovidTokens
training_dataset = qt.create_training_set(list_of_unpreprocessed_documents, list_of_preprocessed_documents)

ctm = CombinedTM(input_size=len(qt.vocab), bert_input_size=768, n_components=50)

ctm.fit(training_dataset) # run the model

ctm.get_topics()

2061
2061


Some weights of the model checkpoint at /home/sakshamm/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [206100/206100]	Train Loss: 453.9426367850861	Time: 0:00:00.507156: : 100it [00:52,  1.90it/s]


defaultdict(list,
            {0: ['fargo',
              'wells',
              'fraud',
              'driver',
              'vehicle',
              'compromise',
              'longer',
              'booklet',
              'though',
              'overseas'],
             1: ['account',
              'first',
              'payment',
              'transfer',
              'week',
              'credit',
              'process',
              'arrival',
              'furniture',
              'funds'],
             2: ['earn',
              'ridiculous',
              'save',
              'advertising',
              'forklift',
              'generate',
              'caller',
              'lensa',
              'debra',
              'grammar'],
             3: ['promise',
              'anyone',
              'steal',
              'inspect',
              'indeed',
              'together',
              'portal',
              'light',
              'refuse',
           